**[MST-01]** モジュールをインポートして、乱数のシードを設定します。

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(20160612)
tf.set_random_seed(20160612)

**[MST-02]** MNISTのデータセットを用意します。

In [2]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
### TensorFlowのバージョンによっては下記のwarningが表示されるが、無視するしかない。
# WARNING:tensorflow:From <ipython-input-2-******>:1: ...
# Instructions for updating:
# Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
#...

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


**[MST-03]** 単層ニューラルネットワークを表現するクラスを定義します。

In [3]:
### Python3.6用に大幅変更
#特にtf.summary関連
class SingleLayerNetwork:
    def __init__(self, num_units):
        with tf.Graph().as_default():
            self.prepare_model(num_units)
            self.prepare_session()

    def prepare_model(self, num_units):
        with tf.name_scope('input'):
            x = tf.placeholder(tf.float32, [None, 784], name='input')
        
        with tf.name_scope('hidden'):
            w1 = tf.Variable(tf.truncated_normal([784, num_units]),
                             name='weights')        
            b1 = tf.Variable(tf.zeros([num_units]), name='biases')
            hidden1 = tf.nn.relu(tf.matmul(x, w1) + b1, name='hidden1')
        
        with tf.name_scope('output'):
            w0 = tf.Variable(tf.zeros([num_units, 10]), name='weights')
            b0 = tf.Variable(tf.zeros([10]), name='biases')
            p = tf.nn.softmax(tf.matmul(hidden1, w0) + b0, name='softmax')

        with tf.name_scope('optimizer'):
            t = tf.placeholder(tf.float32, [None, 10], name='labels')
            loss = -tf.reduce_sum(t * tf.log(p), name='loss')
            train_step = tf.train.AdamOptimizer().minimize(loss)

        with tf.name_scope('evaluator'):
            correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction,
                                              tf.float32), name='accuracy')

        tf.summary.scalar("loss", loss)
        tf.summary.scalar("accuracy", accuracy)
        tf.summary.histogram("weights_hidden", w1)
        tf.summary.histogram("biases_hidden", b1)
        tf.summary.histogram("weights_output", w0)
        tf.summary.histogram
        self.x, self.t, self.p = x, t, p
        self.train_step = train_step
        self.loss = loss
        self.accuracy = accuracy
            
    def prepare_session(self):
        ### >>> with GPU 80%
        #gpuConfig = tf.ConfigProto(
        #    gpu_options = tf.GPUOptions(
        #        per_process_gpu_memory_fraction = 0.8),
        #    device_count = {'GPU':0})
        #sess = tf.Session(config = gpuConfig)
        ### <<<
        #
        ### >>> with CPU only
        sess = tf.Session()
        ### <<<
        sess.run(tf.global_variables_initializer())
        summary = tf.summary.merge_all()
        writer = tf.summary.FileWriter("/tmp/mnist_sl_logs", sess.graph)
        
        self.sess = sess
        self.summary = summary
        self.writer = writer

**[MST-04]** TensorBoard用のデータ出力ディレクトリーを削除して初期化しておきます。

In [4]:
### Linuxの場合、下記の行頭に「#」を挿入し、Windowsの場合「#」を削除
!del /q \tmp\mnist_sl_logs
### Windowsの場合、下記の行頭に「#」を挿入し、 Linuxの場合「#」を削除
#!rm -rf /tmp/mnist_sl_logs

**[MST-05]** パラメーターの最適化を2000回繰り返します。テストセットに対して、約97%の正解率が得られます。

In [5]:
nn = SingleLayerNetwork(1024)

i = 0
for _ in range(2000):
    i += 1
    batch_xs, batch_ts = mnist.train.next_batch(100)
    nn.sess.run(nn.train_step, feed_dict={nn.x: batch_xs, nn.t: batch_ts})
    if i % 100 == 0:
        summary, loss_val, acc_val = nn.sess.run(
            [nn.summary, nn.loss, nn.accuracy],
            feed_dict={nn.x:mnist.test.images, nn.t: mnist.test.labels})
        print ('Step: %d, Loss: %f, Accuracy: %f'
               % (i, loss_val, acc_val))
        nn.writer.add_summary(summary, i)

Step: 100, Loss: 2821.430176, Accuracy: 0.915500
Step: 200, Loss: 2351.062744, Accuracy: 0.931000
Step: 300, Loss: 2009.348755, Accuracy: 0.938600
Step: 400, Loss: 1866.479004, Accuracy: 0.940800
Step: 500, Loss: 1501.254150, Accuracy: 0.955400
Step: 600, Loss: 1417.961914, Accuracy: 0.958800
Step: 700, Loss: 1340.349609, Accuracy: 0.959900
Step: 800, Loss: 1316.767578, Accuracy: 0.961200
Step: 900, Loss: 1298.585449, Accuracy: 0.960600
Step: 1000, Loss: 1237.332275, Accuracy: 0.962000
Step: 1100, Loss: 1317.074463, Accuracy: 0.960900
Step: 1200, Loss: 1109.878662, Accuracy: 0.967100
Step: 1300, Loss: 1159.223877, Accuracy: 0.966600
Step: 1400, Loss: 1138.137939, Accuracy: 0.966200
Step: 1500, Loss: 1099.624268, Accuracy: 0.968100
Step: 1600, Loss: 1102.185547, Accuracy: 0.968900
Step: 1700, Loss: 967.888550, Accuracy: 0.972600
Step: 1800, Loss: 939.744019, Accuracy: 0.973000
Step: 1900, Loss: 1117.150391, Accuracy: 0.969200
Step: 2000, Loss: 1089.584229, Accuracy: 0.968500
